In [1]:
import dask.array as da

In [2]:
import ag2, ag3

ag2_release = ag2.release_data()
ag3_release = ag3.release_data()

/opt/conda/lib/python3.7/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import pandas as pd
import numpy as np

In [4]:
def load_accessibility(chrom, mask_id='gamb_colu'):
    
    if mask_id == "phase2":

        phase2_is_accessible = ag2_release.load_is_accessible(chrom)
        filter_n = ag2_release.load_filter_n(chrom)
        phase2_is_accessible_nonN = da.compress(
            ~filter_n, phase2_is_accessible, axis=0)

        # Compute chunk sizes avoids this error when c
        # concatenating arrays: ValueError: Arrays chunk sizes are unknown: (nan,)
        phase2_is_accessible_nonN.compute_chunk_sizes() 
        return phase2_is_accessible_nonN
    
    else:
        return ag3_release.load_mask(chrom, mask_id=mask_id)

## For given genomic regions, summarize accessibility

### What about "N" sites?

N sites are currently NOT included. 

If a feature contains many N sites- these are not considered in the % calculation.


In [5]:
features =  """
2La     inv     2L      20528089        42165182
2Rb     inv     2R      19444433        26313071
2Rj     inv     2R      3262186 15750717
2Rc     inv     2R      26780000        31450000
2Ru     inv     2R      31480000        35500000
2Rd     inv     2R      33575891        41360919
2Rk     inv     2R      25146360        30717395
CHX     chro    X       20009764        24393108
CH2R    chro    2R      58984778        61545105
CH2L    chro    2L      1       2431617
PEU2L   chro    2L      2487770 5042389
IH2L    chro    2L      5078962 5788875
IH3R    chro    3R      38988757        41860198
CH3R    chro    3R      52161877        53200684
CH3L    chro    3L      1       1815119
PEU3L   chro    3L      1896830 4235209
IH3L    chro    3L      4264713 5031692
"""
# via Thiago Antao

In [6]:
features_df = pd.DataFrame(
    [x.split() for x in features.strip().split("\n")], columns=["id", "type", "seq_id", "start", "stop"])

features_df.start = features_df.start.astype(np.int32)
features_df.stop = features_df.stop.astype(np.int32)
    
    #dtype={"start": np.int32, "stop": np.int32})

In [7]:
features_df["type"] = features_df.type.map({"inv": "inversion", "chro": "chromatin"})

In [8]:
chromosomes = ["2R", "2L", "3R", "3L", "X"]

In [9]:
avail_masks = ["gamb_colu", "arab", "gamb_colu_arab", "phase2"]

In [10]:
features_by_chrom = features_df.groupby("seq_id")

In [11]:
import allel

In [12]:
out = pd.DataFrame(columns=features_df.columns)

In [13]:
out_list = []

for chrom, df in features_by_chrom:
    
    pos = allel.SortedIndex(ag3_release.load_variants(chrom).compute())
    
    for mask_name in avail_masks:
        
        mask = load_accessibility(chrom, mask_name)
        assert mask.shape == pos.shape
        
        for _, row in df.iterrows():
            
            ix = pos.locate_range(row.start, row.stop)
            
            # computing the mean is ok- as it allows us to silently handle Ns, which are ignored.
            access = mask[ix].mean().compute()
            out_list.append([mask_name, chrom, row.id, row.type, row.start, row.stop, access])
            
        out_list.append([mask_name, chrom, chrom, "contig", 1, pos[-1], mask.mean().compute()])

In [14]:
result_df = pd.DataFrame(out_list, columns=["mask_id", "chrom", "feature_name", "type", "start", "stop", "frac_accessible"])

In [15]:
result_df["type"] = pd.Categorical(result_df.type, ["contig", "inversion", "chromatin"], ordered=True)

In [16]:
result_df.sort_values(["mask_id", "type", "chrom", "start"], inplace=True)

In [17]:
result_df.head()

,mask_id,chrom,feature_name,type,start,stop,frac_accessible
9,arab,2L,2L,contig,1,49364325,0.725773
35,arab,2R,2R,contig,1,61545105,0.735970
59,arab,3L,3L,contig,1,41963435,0.696811
73,arab,3R,3R,contig,1,53200684,0.697803
83,arab,X,X,contig,1,24393108,0.533184


In [18]:
for label, df in result_df.groupby("mask_id"):
    df.drop("mask_id", axis=1).to_csv(f"../content/tables/accessibility/accessibility_features_{label}.csv", float_format="%.3f", index=False)

In [19]:
output_df = result_df.pivot(
    index=["chrom", "feature_name", "type", "start", "stop"], 
    columns="mask_id", 
    values="frac_accessible").reset_index()
output_df.index.name = ""

In [20]:
output_df.to_csv(
    "../content/tables/accessibility/table4_accessibility_features_by_mask.csv")

In [21]:
output_df

mask_id,chrom,feature_name,type,start,stop,arab,gamb_colu,gamb_colu_arab,phase2
,,,,,,,,,
0,2L,2L,contig,1,49364325,0.725773,0.741980,0.670365,0.580992
1,2L,2La,inversion,20528089,42165182,0.753209,0.765582,0.701509,0.511216
2,2L,CH2L,chromatin,1,2431617,0.298523,0.377953,0.225203,0.200037
3,2L,IH2L,chromatin,5078962,5788875,0.338672,0.341595,0.264138,0.246504
4,2L,PEU2L,chromatin,2487770,5042389,0.642531,0.643251,0.575857,0.575392
5,2R,2R,contig,1,61545105,0.735970,0.739031,0.674539,0.657463
6,2R,2Rb,inversion,19444433,26313071,0.809948,0.830475,0.759510,0.734145
7,2R,2Rc,inversion,26780000,31450000,0.801247,0.815872,0.745410,0.741728
8,2R,2Rd,inversion,33575891,41360919,0.716491,0.693729,0.635669,0.615729


## Over the exome

In [22]:
exon_data = {}
wg_data = {}

In [23]:
gff_geneset = "../../Anopheles-gambiae-PEST_BASEFEATURES_AgamP4.12.gff3.gz"

In [24]:
for chrom in chromosomes:
    
    exons = gene_features = allel.gff3_to_dataframe(
        gff_geneset, 
        attributes=["ID"]).query("(seqid == @chrom) & (type == 'exon')")
    
    pos = allel.SortedIndex(ag3_release.load_variants(chrom).compute())    
    
    for mask_name in avail_masks:
        
        mask = load_accessibility(chrom, mask_name).compute()
        
        locv, _ = pos.locate_intersection_ranges(exons.start, exons.end)
        
        v = np.compress(locv, mask, axis=0)
        
        exon_data[mask_name, chrom] = pd.Series([v.sum(), locv.sum()], index=["n_accessible", "n_total_bases"], dtype=np.int32)
        
        wg_data[mask_name, chrom] = pd.Series([mask.sum(), mask.shape[0]], index=["n_accessible", "n_total_bases"], dtype=np.int32)

## whole exome

In [25]:
exon_counts = pd.DataFrame.from_dict(exon_data, orient="index")
exon_counts.index.names = ["mask_id", "chrom"]
exon_counts

,,n_accessible,n_total_bases
mask_id,chrom,,
gamb_colu,2R,8326769,9434770
arab,2R,8251192,9434770
gamb_colu_arab,2R,7796125,9434770
phase2,2R,8327539,9434770
gamb_colu,2L,6718168,7423546
arab,2L,6578902,7423546
gamb_colu_arab,2L,6284272,7423546
phase2,2L,6373764,7423546
gamb_colu,3R,6221607,7043977


In [26]:
whole_exome_sum = exon_counts.groupby(level=0).agg(sum)
whole_exome_sum["frac_accessible"] = whole_exome_sum["n_accessible"] / whole_exome_sum["n_total_bases"]
whole_exome_sum

,n_accessible,n_total_bases,frac_accessible
mask_id,,,
arab,28356983,32901366,0.861879
gamb_colu,28891634,32901366,0.878129
gamb_colu_arab,26773478,32901366,0.813750
phase2,28482696,32901366,0.865699


## whole genome

In [27]:
wg_counts = pd.DataFrame.from_dict(wg_data, orient="index")
wg_counts.index.names = ["mask_id", "chrom"]
wg_counts

,,n_accessible,n_total_bases
mask_id,chrom,,
gamb_colu,2R,44439759,60132453
arab,2R,44255681,60132453
gamb_colu_arab,2R,40561667,60132453
phase2,2R,39534863,60132453
gamb_colu,2L,36005131,48525747
arab,2L,35218697,48525747
gamb_colu_arab,2L,32529983,48525747
phase2,2L,28193050,48525747
gamb_colu,3R,37199402,52226568


In [28]:
whole_genome_sum = wg_counts.groupby(level=0).agg(sum)
whole_genome_sum["frac_accessible"] = whole_genome_sum["n_accessible"] / whole_genome_sum["n_total_bases"]
whole_genome_sum

,n_accessible,n_total_bases,frac_accessible
mask_id,,,
arab,156787860,225028590,0.696746
gamb_colu,162714957,225028590,0.723086
gamb_colu_arab,143569766,225028590,0.638007
phase2,139717733,225028590,0.620889
